# MovieLens with SAP HANA PAL APRIORI


## **Initialize the connection**

In [1]:
import sqlalchemy, os
from sqlalchemy import create_engine

%reload_ext sql
%config SqlMagic.displaylimit = 5
%config SqlMagic.feedback = False
%config SqlMagic.autopandas = True

hxe_connection = 'hana://userkey=HXEHOST';

%sql $hxe_connection

import pandas as pd
pd.options.display.max_rows = 100

from IPython.core.display import HTML
HTML("""
<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 

$( document ).ready(code_toggle);
</script>

To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.""")

## **Cleanup Model Tables and APL objects**

In [2]:
%%sql
-- --------------------------------------------------------------------------
-- drop function in/out tables, helper tables and views
-- --------------------------------------------------------------------------
drop table pal_movielens_parameters;
drop table pal_movielens_apriori_pmmlmodel;
drop table pal_movielens_apriori_result;
drop  view pal_movielens_apriori_data_input;

 * hana://userkey=HXEHOST


""


## **Create Model Specific table **

In [3]:
%%sql
-- --------------------------------------------------------------------------
-- create the config and output tables
-- --------------------------------------------------------------------------
create column table pal_movielens_parameters (
	param_name   varchar(100),
	int_value    integer,
	double_value double,
	string_value varchar (100)
);

create column table pal_movielens_apriori_result (
	prerule    varchar(500),
	postrule   varchar(500),
	support    double,
	confidence double,
	lift       double
);
create row table pal_movielens_apriori_pmmlmodel (
  row_index integer,
  model_content clob
);
-- --------------------------------------------------------------------------
-- create the input data view
-- --------------------------------------------------------------------------
create view pal_movielens_apriori_data_input as
select userid, movieid
from   movielens_ratings;

 * hana://userkey=HXEHOST


""


## **Run the Recommendation algorithm**

In [4]:
%%sql
-- --------------------------------------------------------------------------
-- configuration
-- --------------------------------------------------------------------------
truncate table pal_movielens_parameters;
insert into pal_movielens_parameters values ('MIN_SUPPORT'   , null, 0.1 , null); -- no default
insert into pal_movielens_parameters values ('MIN_CONFIDENCE', null, 0.1 , null); -- no default
insert into pal_movielens_parameters values ('MIN_LIFT'      , null, 0.0 , null); -- default is 0.0
insert into pal_movielens_parameters values ('MAX_CONSEQUENT', 1   , null, null); -- default is 500
insert into pal_movielens_parameters values ('MAXITEMLENGTH' , 2   , null, null); -- default is 5
insert into pal_movielens_parameters values ('UBIQUITOUS'    , null, 1.0 , null); -- default is 1.0
insert into pal_movielens_parameters values ('PMML_EXPORT'   , 1   , null, null); -- default is 0
insert into pal_movielens_parameters values ('TIMEOUT'       , 3600, null, null); -- default is 3600
insert into pal_movielens_parameters values ('THREAD_RATIO'  , null, 0.0 , null); -- default is 0.0
select * from pal_movielens_parameters;

 * hana://userkey=HXEHOST


,param_name,int_value,double_value,string_value
0,MIN_SUPPORT,NaN,0.1,None
1,MIN_CONFIDENCE,NaN,0.1,None
2,MIN_LIFT,NaN,0.0,None
3,MAX_CONSEQUENT,1.0,NaN,None
4,MAXITEMLENGTH,2.0,NaN,None
5,UBIQUITOUS,NaN,1.0,None
6,PMML_EXPORT,1.0,NaN,None
7,TIMEOUT,3600.0,NaN,None
8,THREAD_RATIO,NaN,0.0,None


In [5]:
%%sql
truncate table pal_movielens_apriori_result;
truncate table pal_movielens_apriori_pmmlmodel;
-- --------------------------------------------------------------------------
-- execute the pal function to train the model
-- --------------------------------------------------------------------------
call _sys_afl.pal_apriori(
    pal_movielens_apriori_data_input
  , pal_movielens_parameters
  , pal_movielens_apriori_result
  , pal_movielens_apriori_pmmlmodel
) with overview;

 * hana://userkey=HXEHOST


,variable,table
0,P3,"""ML_USER"".""PAL_MOVIELENS_APRIORI_RESULT"""
1,P4,"""ML_USER"".""PAL_MOVIELENS_APRIORI_PMMLMODEL"""


## **Check the output**

In [6]:
%sql select * from pal_movielens_apriori_result;

 * hana://userkey=HXEHOST


,prerule,postrule,support,confidence,lift
0,150,208,0.123696,0.415000,2.464292
1,208,150,0.123696,0.734513,2.464292
2,165,208,0.111773,0.528169,3.136296
3,208,165,0.111773,0.663717,3.136296
4,592,208,0.128167,0.438776,2.605472
5,208,592,0.128167,0.761062,2.605472
6,356,208,0.144560,0.284457,1.689124
7,208,356,0.144560,0.858407,1.689124
8,593,208,0.125186,0.276316,1.640778
9,208,593,0.125186,0.743363,1.640778


In [7]:
%sql select * from pal_movielens_apriori_pmmlmodel;

 * hana://userkey=HXEHOST


,row_index,model_content
0,1,"<PMML version=""4.0"" xmlns=""http://www.dmg.org/..."


## **Collaborative filtering - Create the result view**

In [8]:
%sql drop   view pal_movielens_apriori_result_collaborative;

 * hana://userkey=HXEHOST


""


In [9]:
%%sql
create view pal_movielens_apriori_result_collaborative as
select *
from (
  select
      t1.userid
    , row_number() over(partition by t1.userid order by t1.score desc, t1.consequent desc ) as rank
    , t1.consequent as movieid
    , t1.score
    , movies.title
    , movies.genres
    , links.imdbid
    , links.tmdbid
  from (
    select input_data.userid, rules.postrule as consequent, max(rules.confidence) as score
    from movielens_ratings as input_data
    left outer join (select * from pal_movielens_apriori_result) rules on (cast (input_data.movieid as varchar(500)) = rules.prerule)
    where rules.postrule is not null
    group by input_data.userid, rules.postrule
  ) t1
    left outer join movielens_movies movies on movies.movieid = t1.consequent
    left outer join movielens_links  links  on links.movieid  = t1.consequent
) t1
where t1.rank <= 5;

 * hana://userkey=HXEHOST


""


## **Content-based filtering - Create the result view**

In [10]:
%sql drop   view pal_movielens_apriori_result_contentbased;

 * hana://userkey=HXEHOST


""


In [11]:
%%sql
create view pal_movielens_apriori_result_contentbased as
select *
from (
  select
      t1.movieid
    , row_number() over(partition by t1.movieid order by t1.score desc, t1.consequent desc ) as rank
    , t1.consequent as similar_movieid
    , t1.score
    , movies.title
    , movies.genres
    , links.imdbid
    , links.tmdbid
  from (
    select movieid, rules.postrule as consequent, rules.confidence as score
    from movielens_movies as input_data
    left outer join (select * from pal_movielens_apriori_result) rules on (cast (input_data.movieid as varchar(500)) = rules.prerule)
    where rules.postrule is not null
  ) t1
    left outer join movielens_movies movies on movies.movieid = t1.consequent
    left outer join movielens_links  links  on links.movieid  = t1.consequent
) t1
where t1.rank <= 5;

 * hana://userkey=HXEHOST


""


## **Collaborative filtering - Validate the result**
Let's verify how many users will actually get recommendations using the following SQL:

In [12]:
%%sql
select reco_count, count(1) as user_count
from (
  select userid, max(rank) as reco_count
  from   pal_movielens_apriori_result_collaborative
  group by userid
) group by reco_count order by reco_count desc;

 * hana://userkey=HXEHOST


,reco_count,user_count
0,5,660


Let's verify how many distinct movies will actually get recommended to a user (part of the top 5 scores) using the following SQL:

In [13]:
%%sql
select
    count(1) as movie_count
  , count(1) *100 / (select count(1) as count from movielens_movies ) as movie_ratio
from (
  select movieid
  from   pal_movielens_apriori_result_collaborative
  group by movieid
);

 * hana://userkey=HXEHOST


,movie_count,movie_ratio
0,41,0.449315


Let's verify how many distinct movies will potentially get recommended to a user (not just the top 5 scores) using the following SQL:

In [14]:
%%sql
select
    count(1) as movie_count
  , count(1) *100 / (select count(1) as count from movielens_movies ) as movie_ratio
from (
  select prerule as movieid
  from pal_movielens_apriori_result
  where prerule not like '%&%'
  group by prerule
);

 * hana://userkey=HXEHOST


,movie_count,movie_ratio
0,200,2.19178


Only 660 of the initial users will receive the requested 5 recommendations. However, only 200 distinct movies (2.2%) in total can be proposed overall and 41 movies (0.45%) in the top 5 lists.

## **Content-based filtering - Validate the result**
Let's verify how many movies will actually get recommendations using the following SQL:

In [15]:
%%sql
select reco_count, count(1) as movie_count
from (
  select movieid, max(rank) as reco_count
  from pal_movielens_apriori_result_contentbased
  group by movieid
) group by reco_count order by 1 desc;

 * hana://userkey=HXEHOST


,reco_count,movie_count
0,5,166
1,4,5
2,3,7
3,2,8
4,1,14


Let's verify how many distinct similar movies will actually get recommended to a movie (part of the top 5 scores) using the following SQL:

In [16]:
%%sql
select
    count(1) as movie_count
  , count(1) *100 / (select count(1) as count from movielens_movies ) as movie_ratio
from (
  select movieid
  from pal_movielens_apriori_result_contentbased
  group by movieid
);

 * hana://userkey=HXEHOST


,movie_count,movie_ratio
0,200,2.19178


Let's verify how many rating does the movies with no recommendation have using the following SQL:

In [17]:
%%sql
select rating_count, count(1) as movie_count
from (
  select ratings.movieid, count(1) as rating_count
  from movielens_ratings ratings
  left outer join (
    select movieid
    from (
      select prerule as movieid
      from pal_movielens_apriori_result
      where prerule not like '%&%'
      group by prerule
    )
  ) t1 on (ratings.movieid = t1.movieid)
  where t1.movieid is null
  group by ratings.movieid
) group by rating_count;

 * hana://userkey=HXEHOST


,rating_count,movie_count
0,1,3063
1,2,1202
2,3,755
3,4,550
4,5,397
5,6,305
6,7,206
7,8,187
8,9,156
9,10,162


As you can see, the movies with no recommendations have up to 92 ratings, and this list include the 3063 movies with only one rating and the 1202 with only 2 ratings.